In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_all = pd.read_csv('../data/data.csv')

train, test = train_test_split(data_all, test_size=0.2, random_state=42)

X_train = train.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_train = train['DEFAULT']

X_test = test.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_test = test['DEFAULT']

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin

class OutliersReplacer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for column in X.columns:
            if column == 'SAVINGS':
                X.loc[X[column] > 2500000, column] = 2500000
            elif column == 'DEBT':
                X.loc[X[column] > 4000000, column] = 4000000
            elif column == 'T_CLOTHING_12':
                X.loc[X[column] > 32000, column] = 32000
            elif column == 'T_CLOTHING_6':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_12':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_6':
                X.loc[X[column] > 18000, column] = 18000
            elif column == 'T_TRAVEL_12':
                X.loc[X[column] > 150000, column] = 150000
            elif column == 'T_TRAVEL_6':
                X.loc[X[column] > 110000, column] = 110000
        return X

In [3]:
class DropColumns(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        potentialColumnsToDrop = ['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12', 'T_EDUCATION_6','T_ENTERTAINMENT_6','T_GAMBLING_6','T_GROCERIES_6','T_HOUSING_6','T_EXPENDITURE_12', 'T_EXPENDITURE_6','R_GROCERIES_DEBT', 'INCOME', 'T_UTILITIES_6', 'R_EDUCATION_DEBT', 'T_UTILITIES_12', 'R_CLOTHING_DEBT',
        'CAT_DEPENDENTS', 'R_ENTERTAINMENT_SAVINGS', 'R_FINES_INCOME',
        'R_FINES_SAVINGS', 'R_FINES_DEBT', 'R_GROCERIES_SAVINGS',
        'CAT_SAVINGS_ACCOUNT', 'R_HOUSING_INCOME', 'R_TAX_INCOME',
        'R_TAX_SAVINGS', 'R_TRAVEL_DEBT', 'R_UTILITIES_DEBT', 'CAT_GAMBLING',
        'CAT_DEBT', 'CAT_MORTGAGE', 'SAVINGS', 'R_UTILITIES_SAVINGS', 'R_EDUCATION', 'R_FINES', 'R_GAMBLING', 'R_HOUSING', 'R_GROCERIES_INCOME', 'T_ENTERTAINMENT_12', 'R_ENTERTAINMENT',
       'R_TRAVEL_SAVINGS', 'R_GAMBLING_SAVINGS', 'T_CLOTHING_6', 'CUST_ID']
        for column in potentialColumnsToDrop:
            if column in X.columns:
                X.drop(column, axis=1, inplace=True)
        return X

In [45]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for n_est in range(13, 24, 1):
    for depth in range(2, 4, 1):
        for lr in [0.1, 0.2, 0.3]:
            pipeline = Pipeline([
                ('outliers_replacer', OutliersReplacer()),
                ('drop_columns', DropColumns()),
                ('classifier', GradientBoostingClassifier(n_estimators=n_est, max_depth=depth, learning_rate=lr))
            ])
            kfold = KFold(n_splits=5, random_state=42, shuffle=True)
            score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
            score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
            score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
            score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
            print(f"n_estimators: {n_est}, max_depth: {depth}, learning_rate: {lr}, accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")


n_estimators: 13, max_depth: 2, learning_rate: 0.1, accuracy: 0.7265625, recall: 0.10737262737262739, precision: 0.6444444444444445, f1: 0.17964768262556124
n_estimators: 13, max_depth: 2, learning_rate: 0.2, accuracy: 0.7203125, recall: 0.14758574758574758, precision: 0.4991666666666667, f1: 0.2302491565731984
n_estimators: 13, max_depth: 2, learning_rate: 0.3, accuracy: 0.721875, recall: 0.16833832833832835, precision: 0.49162846956964606, f1: 0.24512146253850378
n_estimators: 13, max_depth: 3, learning_rate: 0.1, accuracy: 0.721875, recall: 0.1358108558108558, precision: 0.5822222222222222, f1: 0.21166928309785452
n_estimators: 13, max_depth: 3, learning_rate: 0.2, accuracy: 0.6953125, recall: 0.14665334665334667, precision: 0.3696650717703349, f1: 0.2000788231376467
n_estimators: 13, max_depth: 3, learning_rate: 0.3, accuracy: 0.7015625, recall: 0.22965034965034964, precision: 0.4218081435472739, f1: 0.30059411852515294
n_estimators: 14, max_depth: 2, learning_rate: 0.1, accuracy: 

In [30]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for n_est in range(12, 30, 3):
    for depth in range(2, 4, 1):
        for lr in [ 0.1, 0.5]:
            pipeline = Pipeline([
                ('outliers_replacer', OutliersReplacer()),
                ('drop_columns', DropColumns()),
                ('classifier', GradientBoostingClassifier(n_estimators=n_est, max_depth=depth, learning_rate=lr))
            ])
            kfold = KFold(n_splits=10, random_state=42, shuffle=True)
            score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
            score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
            score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
            score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
            print(f"n_estimators: {n_est}, max_depth: {depth}, learning_rate: {lr}, accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")


n_estimators: 12, max_depth: 2, learning_rate: 0.1, accuracy: 0.7265625, recall: 0.10220182552225895, precision: 0.6958333333333333, f1: 0.1695680779488829
n_estimators: 12, max_depth: 2, learning_rate: 0.5, accuracy: 0.7078125, recall: 0.22980040083755254, precision: 0.45125180375180374, f1: 0.3141417581645286
n_estimators: 12, max_depth: 3, learning_rate: 0.1, accuracy: 0.7203125, recall: 0.11785769313246093, precision: 0.5516666666666666, f1: 0.21462521358631198
n_estimators: 12, max_depth: 3, learning_rate: 0.5, accuracy: 0.7015625, recall: 0.21201445312513423, precision: 0.38025668775668775, f1: 0.3163471089961521
n_estimators: 15, max_depth: 2, learning_rate: 0.1, accuracy: 0.7234375, recall: 0.11172563504606847, precision: 0.6266666666666667, f1: 0.17949493002124578
n_estimators: 15, max_depth: 2, learning_rate: 0.5, accuracy: 0.7109375, recall: 0.25563889412960616, precision: 0.5144300144300143, f1: 0.33161433338630225
n_estimators: 15, max_depth: 3, learning_rate: 0.1, accurac

In [6]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

pipeline = Pipeline([
        ('outliers_replacer', OutliersReplacer()),
        ('drop_columns', DropColumns()),
        ('classifier', GradientBoostingClassifier(n_estimators=15, max_depth=2, learning_rate=0.1))
    ])


#validation 
kfold = KFold(n_splits=10, random_state=42, shuffle=True)
score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
print(f"accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")


accuracy: 0.73125, recall: 0.12790390122433465, precision: 0.6788888888888889, f1: 0.2043432700770162


In [7]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.74375
[[113   1]
 [ 40   6]]
              precision    recall  f1-score   support

           0       0.74      0.99      0.85       114
           1       0.86      0.13      0.23        46

    accuracy                           0.74       160
   macro avg       0.80      0.56      0.54       160
weighted avg       0.77      0.74      0.67       160

